In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Linear Regression (House Price Prediction)


# Import Important Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score , r2_score , mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# Data Understanding

In [ ]:
house_data = pd.read_csv("/kaggle/input/california-housing-prices/housing.csv")
print(house_data.shape)
house_data.head(10)

In [ ]:
house_data.sample(5)

In [ ]:
house_data.info()

In [ ]:
house_data.describe()

In [ ]:
bins = np.arange(0 , house_data["housing_median_age"].max() + 1 , 1)
plt.hist(data = house_data , x = "housing_median_age" , bins = bins);

In [ ]:
house_data["housing_median_age"].skew()

In [ ]:
bins = np.arange(0 , house_data["median_income"].max() + 0.3 , 0.3)
plt.hist(data = house_data , x = "median_income" , bins = bins);

In [ ]:
bins = np.arange(0 , house_data["median_house_value"].max() + 10000 , 10000)
plt.hist(data = house_data , x = "median_house_value" , bins = bins);

In [ ]:
bins = np.arange(0 , house_data["total_rooms"].max() + 200 , 200)
plt.hist(data = house_data , x = "total_rooms" , bins = bins);

In [ ]:
house_data["total_rooms"].skew()

In [ ]:
bins = np.arange(0 , house_data["total_bedrooms"].max() + 60 , 60)
plt.hist(data = house_data , x = "total_bedrooms" , bins = bins);

In [ ]:
bins = np.arange(0 , house_data["population"].max() + 280 , 280)
plt.hist(data = house_data , x = "population" , bins = bins);

In [ ]:
bins = np.arange(0 , house_data["households"].max() + 45 , 45)
plt.hist(data = house_data , x = "households" , bins = bins);

In [ ]:
bins = np.arange(0 , house_data["latitude"].max() + 0.6 , 0.6)
plt.hist(data = house_data , x = "latitude" , bins = bins);

# Data Cleaning and Preprocessing

### Hundle Missing Vlaues


In [ ]:
print(house_data.isnull().sum() , "\n")
percentage = house_data.isnull().sum().sum() / house_data.shape[0] * 200
print(percentage)

##### Number of null rows is very small compare to all rows so we will drop it

In [ ]:
house_data.dropna(inplace = True)
house_data.isnull().sum()

### Hundle Outliers

In [ ]:
plt.figure(figsize = [20,10])
plt.subplot(1 , 2 , 1)

bins = np.arange(0 , house_data["total_rooms"].max() + 200 , 200)
plt.hist(data = house_data , x = "total_rooms" , bins = bins)
plt.xticks(np.arange(0 , 40000 , 1000))
plt.xticks(rotation = 90);

plt.subplot(1 , 2 , 2)

sns.boxplot(x = house_data["total_rooms"])

In [ ]:
out_index = house_data[house_data["total_rooms"] > 20000].index
house_data = house_data.drop(out_index , axis = 0) 
plt.figure(figsize = [20,10])
plt.subplot(1 , 2 , 1)

bins = np.arange(0 , house_data["total_rooms"].max() + 200 , 200)
plt.hist(data = house_data , x = "total_rooms" , bins = bins)
plt.xticks(np.arange(0 , 40000 , 1000))
plt.xticks(rotation = 90);

plt.subplot(1 , 2 , 2)

sns.boxplot(x = house_data["total_rooms"])
house_data["total_rooms"].describe()

In [ ]:
plt.figure(figsize = [20,10])
plt.subplot(1 , 2 , 1)

bins = np.arange(0 , house_data["total_bedrooms"].max() + 80 , 80)
plt.hist(data = house_data , x = "total_bedrooms" , bins = bins)
plt.xticks(np.arange(0 , 6000 , 150))
plt.xticks(rotation = 90);

plt.subplot(1 , 2 , 2)

sns.boxplot(x = house_data["total_bedrooms"])

In [ ]:
out_index2 = house_data[house_data["total_bedrooms"] > 3400].index
house_data = house_data.drop(out_index2 , axis = 0) 
plt.figure(figsize = [20,10])
plt.subplot(1 , 2 , 1)

bins = np.arange(0 , house_data["total_bedrooms"].max() + 80 , 80)
plt.hist(data = house_data , x = "total_bedrooms" , bins = bins)
plt.xticks(np.arange(0 , 6000 , 150))
plt.xticks(rotation = 90);

plt.subplot(1 , 2 , 2)

sns.boxplot(x = house_data["total_bedrooms"])
house_data["total_bedrooms"].describe()


In [ ]:
plt.figure(figsize = [20,10])
plt.subplot(1 , 2 , 1)

bins = np.arange(0 , house_data["population"].max() + 200 , 200)
plt.hist(data = house_data , x = "population" , bins = bins)
plt.xticks(np.arange(0 , 35000 , 1000))
plt.xticks(rotation = 90);

plt.subplot(1 , 2 , 2)

sns.boxplot(x = house_data["population"])
plt.xticks(np.arange(0 , 35000 , 1000))
plt.xticks(rotation = 90);

In [ ]:
out_index3 = house_data[house_data["population"] > 9000].index
house_data = house_data.drop(out_index3 , axis = 0) 
plt.figure(figsize = [20,10])
plt.subplot(1 , 2 , 1)

bins = np.arange(0 , house_data["population"].max() + 200 , 200)
plt.hist(data = house_data , x = "population" , bins = bins)
plt.xticks(np.arange(0 , 35000 , 1000))
plt.xticks(rotation = 90);

plt.subplot(1 , 2 , 2)

sns.boxplot(x = house_data["population"])
plt.xticks(np.arange(0 , 35000 , 1000))
plt.xticks(rotation = 90);house_data["population"].describe()

In [ ]:
plt.figure(figsize = [20,10])
plt.subplot(1 , 2 , 1)

bins = np.arange(0 , house_data["households"].max() + 40 , 40)
plt.hist(data = house_data , x = "households" , bins = bins)
plt.xticks(np.arange(0 , 6500 , 150))
plt.xticks(rotation = 90);

plt.subplot(1 , 2 , 2)
sns.boxplot(x = house_data["households"]);

In [ ]:
out_index4 = house_data[house_data["households"] > 3150].index
house_data = house_data.drop(out_index4 , axis = 0) 
plt.figure(figsize = [20,10])
plt.subplot(1 , 2 , 1)

bins = np.arange(0 , house_data["households"].max() + 40 , 40)
plt.hist(data = house_data , x = "households" , bins = bins)
plt.xticks(np.arange(0 , 6500 , 150))
plt.xticks(rotation = 90);

plt.subplot(1 , 2 , 2)
sns.boxplot(x = house_data["households"]);house_data["households"].describe()

In [ ]:
plt.figure(figsize = [20,10])
plt.subplot(1 , 2 , 1)

bins = np.arange(0 , house_data["median_income"].max() + 0.2 , 0.2)
plt.hist(data = house_data , x = "median_income" , bins = bins);

plt.subplot(1 , 2 , 2)

sns.boxplot(x = house_data["median_income"])

In [ ]:
out_index5 = house_data[house_data["median_income"] > 14].index
house_data = house_data.drop(out_index5 , axis = 0) 
plt.figure(figsize = [20,10])
plt.subplot(1 , 2 , 1)

bins = np.arange(0 , house_data["median_income"].max() + 0.2 , 0.2)
plt.hist(data = house_data , x = "median_income" , bins = bins);

plt.subplot(1 , 2 , 2)

sns.boxplot(x = house_data["median_income"])

In [ ]:
plt.figure(figsize = [20,10])
plt.subplot(1 , 2 , 1)

bins = np.arange(0 , house_data["median_house_value"].max() + 5300 , 5300)
plt.hist(data = house_data , x = "median_house_value" , bins = bins);

plt.subplot(1 , 2 , 2)

sns.boxplot(x = house_data["median_house_value"])

### Check For duplicated

In [ ]:
house_data.duplicated().sum()

In [ ]:
house_data["ocean_proximity"].value_counts()

### Feature Engineering

In [ ]:
house_data = house_data.join(pd.get_dummies(house_data.ocean_proximity).astype(int)).drop(["ocean_proximity"], axis=1)

In [ ]:
house_data["bedrooms_ratio"] = house_data["total_bedrooms"] / house_data["total_rooms"]
house_data["households_rooms"] = house_data["total_rooms"] / house_data["households"]

# Data Exploration (EDA)

In [ ]:
house_data.head(10)

In [ ]:
house_data.corr(numeric_only = True)

In [ ]:
plt.scatter(house_data["median_income"], house_data["median_house_value"], c ="blue");

In [ ]:
plt.scatter(house_data["housing_median_age"], house_data["total_rooms"], c ="blue");

In [ ]:
plt.scatter(house_data["housing_median_age"], house_data["population"], c ="blue");

In [ ]:
plt.scatter(house_data["total_rooms"], house_data["population"], c ="blue");

In [ ]:
plt.scatter(house_data["total_rooms"], house_data["median_income"], c ="blue");

In [ ]:
plt.figure(figsize = (15,8))
sns.scatterplot(x = house_data["longitude"] , y = house_data["latitude"] , data = house_data ,hue = "median_house_value" , palette = "coolwarm")

# Linear Regression Model

### Prepare Data

In [ ]:
house_data.describe()

#### Feature Scaling

In [ ]:
scaler = StandardScaler()
house_data[["longitude","latitude","housing_median_age","total_rooms","total_bedrooms","population","households","median_income","bedrooms_ratio","households_rooms"]] = scaler.fit_transform(house_data[["longitude","latitude","housing_median_age","total_rooms","total_bedrooms","population","households","median_income","bedrooms_ratio","households_rooms"]])
house_data.describe()

#### Data Spliting

In [ ]:
y = house_data['median_house_value']  
x = house_data.drop('median_house_value', axis=1) 

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

## Model Training

In [ ]:
model = LinearRegression()
model.fit(x_train, y_train)

y_train_pred = model.predict(x_train)
y_test_pred = model.predict(x_test)

## Evaluate the Model

In [ ]:
rmse_train = mean_squared_error(y_train, model.predict(x_train), squared=False)

rmse_test = mean_squared_error(y_test, model.predict(x_test), squared=False)

r2_train = r2_score(y_train, model.predict(x_train))
r2_test = r2_score(y_test, model.predict(x_test))

print(f"Training RMSE: {rmse_train:.2f}")
print(f"Testing RMSE: {rmse_test:.2f}")
print(f"Training R²: {r2_train:.2f}")
print(f"Testing R²: {r2_test:.2f}")